In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Hannanum
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
from kiwipiepy import Kiwi
import re

han = Hannanum()

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = ['_', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

EP = [
    [['ㅈ','ㅓ','_','ㄴ','ㅡ','ㄴ'],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']]
]

EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','_','ㄷ','ㅏ','_'],['ㄷ','ㅏ','_']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']]
]

NB = [
    
    [['ㅈ','ㅓ','ㄴ'],['ㄴ','ㅏ','ㄴ']],
    [['ㅈ','ㅓ','ㄴ','ㅡ','ㄴ'],['ㄴ','ㅏ','ㄴ','ㅡ','ㄴ']]
    
]
#New function
class Jamodealer:
    jamo = []
    pp = ''
    def __init__(self,lis_word):
        #각 단어들을 받아와서 자모단위로 나눈다.
        print('the jamo starts!')
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
        
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.          
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다. 
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        restored_text = join_jamos(restored_jamo)
        #합쳐진 문장을 return 한다. 
        return restored_text
    
#Not use
def tojamo(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

#Not use, to make one dimension
def to1dim(input):
    result=[]
    for i in input:
        for j in i:
            result.append(j)
    return result

#Not use, to make two dimension
def to2dim(input):
    result = []
    li = []
    for i in input:
        if i == ' ':
            result.append([' '])
        else:
            li.append(i)
        if len(li)==3:
            result.append(li)
            li = []
    return result

#Not use, 기존의 문장을 합치는 function
def makeone(input):
    result = ''
    li = ''
    for i in input:
        if i[0]==' ':
            result = result+' '
        else:
            ind = ord('가')
            ind +=CHOSUNG_LIST.index(i[0])*588
            ind +=JUNGSUNG_LIST.index(i[1])*28
            ind +=JONGSUNG_LIST.index(i[2])
            result = result+chr(ind)
    return result

#list to string         
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

#string to list
def str2li(input):
    li = []
    for i in range(len(input)):
        li.append(input[i])
    return li

#To make Jamo Dictionary
#Dictionary의 요소를 jamo로 바꾸는 함수이다. 
def makejamodict(input):
    result = []
    for i in input:
        bullet = []
        one = []
        two = []
        gre1 = tojamo(i[0])
        for j in gre1:
            for k in j:
                one.append(k)
        bullet.append(one)
        gre2 = tojamo(i[1])
        for j in gre2:
            for k in j:
                two.append(k)
        bullet.append(two)
        result.append(bullet)
    return result

#To make String dictionary
#Convert List element to String
#각 tag 사전의 key와 value가 string단위로 합쳐지게 해서 re.sub() 사용이 가능해진다. 
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        key = li2str(i[0])
        value = li2str(i[1])
        bullet.append(key)
        bullet.append(value)
        result.append(bullet)
    return result

#strlis = makestrdict(lis)
#strlis = makejamodict(lili)
#strlis = makestrdict(strlis)

#
EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NB_dict = makestrdict(NB)

Dict_list=['EP','EF','NB']
Dict_map = [EP_dict, EF_dict, NB_dict]

#divide by morpheme
#형태소로 변환 후 tag와 word를 따로 저장한다. 이때 각각의 index는 일치한다. 
def to2lists(input):
    lis_word = []
    lis_tag = []
    data = han.pos(input,ntags=22,flatten=True, join=False)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag
    

class Changer(object):
    def __init__(self):
        print('the changer starts!')
        
    def high_low(self, stc):
        result = stc
        lis_word, lis_tag = to2lists(result)
        jam = Jamodealer(lis_word)
        for i in range(len(lis_tag)):
            if lis_tag[i] in Dict_list:ㄸ
                dic = Dict_map[Dict_list.index(lis_tag[i])]
                res = jam.jamo[i]
                for j in range(len(dic)):
                    res = re.sub(dic[j][0],dic[j][1],res)
                jam.jamo[i] = res
                
        print(jam.make_one())
        return jam.make_one()
        
    
    def processText(self,stc):
        result = stc
        res = self.high_low(result)
        return res
    



In [2]:
txt = '전 그것을 좋아합니다'
ch = Changer()
tt = ch.processText(txt)
print(tt)

the changer starts!
the jamo starts!
난그것을좋아한다
난그것을좋아한다


In [3]:
tt

'난그것을좋아한다'

In [4]:
txt = '전 그것을 좋아합니다'
ch = Changer()
tt = ch.processText(txt)
print(tt)

the changer starts!
the jamo starts!
난그것을좋아한다
난그것을좋아한다


In [5]:
EP_dict

[['ㅈㅓ_ㄴㅡㄴ', 'ㄴㅏ_ㄴㅡㄴ'], [' ㅈㅓㄴ ', 'ㄴㅏ_ㄴㅡㄴ'], ['ㅇㅓ_ㅇㅛ_', 'ㄷㅏ_']]